<a href="https://colab.research.google.com/github/Nell87/drivendata_richter/blob/main/script/feature_selection_and_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing data** 

In [22]:
####    INCLUDES  _______________________________________ #### 
#Loading Libraries:# 
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

####    READING TRAIN AND TEST DATA _______________________________________ #### 
train_values = data = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_values.csv")
train_labels = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_labels.csv")
test_values = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/test_values.csv")
train_merge = train_values.merge(train_labels, on = 'building_id', how = 'inner',)
print(train_merge.shape)

(260601, 40)


# **Feature engineering** 

## **Feature engineering*: Location features**
The features **geo_level_1_id, geo_level_2_id, geo_level_3_id** represent the geographic region in which building exists, from largest (level 1) to most specific sub-region (level 3). Possible values: level 1: 0-30, level 2: 0-1427, level 3: 0-12567.

For every location feature there is a high number of categorical values, so we'll apply feature engineering on them. We'll replace every value with their conditional probabilities respect to every damage_grade category

In [100]:
# Function to replace a categorical feature with many values, with their conditional probabilities respecto to the predicted feature
def categoricalvalues_condprob(data, index, pred_feature, new_column_name):
  # Create prob table
  probs = data.groupby(index).size().div(len(data))
  probs_group = data.groupby([index, pred_feature]).size().div(len(data)).div(probs, axis=0, level=index).reset_index()
  probs_group.columns= [index, pred_feature, new_column_name]
  probs_group_wide = probs_group.pivot(index=[index], columns = pred_feature,values = new_column_name) #Reshape from long to wide
  probs_group_wide = probs_group_wide.reset_index()
 
 # Rename columns
  unique_values = np.unique(data[pred_feature])
  unique_values = -(len(unique_values))
  for i in range(unique_values,0):
    probs_group_wide.rename(columns={probs_group_wide.columns[i]: i}, inplace = True)
    
  # Add column to main dataset
  data_merge = data.merge(probs_group_wide, on=index, how='left')

  # Get rid of the categorical feature
  data_merge = data_merge.drop(index, axis=1)

  # Return dataset
  return probs_group_wide

# Apply the function
train_merge2 = categoricalvalues_condprob(train_merge, 'geo_level_1_id', 'damage_grade', 'prob_cond_geo_level_1')
#train_merge = categoricalvalues_condprob(train_merge, 'geo_level_2_id', 'damage_grade', 'prob_cond_geo_level_2')
#train_merge = categoricalvalues_condprob(train_merge, 'geo_level_3_id', 'damage_grade', 'prob_cond_geo_level_3')

train_merge2

damage_grade,geo_level_1_id,-3,-2,-1
0,0,0.084019,0.766642,0.149339
1,1,0.152166,0.734913,0.112921
2,2,0.091300,0.655209,0.253491
3,3,0.032493,0.603448,0.364058
4,4,0.035763,0.766337,0.197900
5,5,0.165799,0.748699,0.085502
6,6,0.086461,0.665354,0.248185
7,7,0.054386,0.593503,0.352111
8,8,0.034277,0.446174,0.519549
9,9,0.141738,0.690500,0.167761


# **Feature selection** 

In [88]:
unique_values = np.unique(train_merge.damage_grade)
unique_values = -(len(unique_values))
unique_values

for i in range(unique_values,0):
   print(i)


-3
-2
-1
